In [1]:
# !pip install transformers
# !pip install git+https://github.com/yuchenlin/LLM-Blender.git
# !pip install "pydantic<2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
# !pip install pip install accelerate

In [2]:
# !pip install pip install accelerate

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import csv
from torch.optim.lr_scheduler import CosineAnnealingLR


# from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:

# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   eos_token='</s>'
# )
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm

# model = AutoModelForCausalLM.from_pretrained("./kakao_aug_2_epoch/")
# tokenizer = AutoTokenizer.from_pretrained("./kakao_aug_2_epoch/", eos_token='</s>')
model = AutoModelForCausalLM.from_pretrained("./kko_aug2_5_epoch/")
tokenizer = AutoTokenizer.from_pretrained("./kko_aug2_5_epoch/", eos_token='</s>',max_length=10000)



Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
test_df = pd.read_csv("./data/test.csv")
# test_df
# test_df["질문"].iloc[107]

In [6]:

appedix_dict_1 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('?') >=2:
                appendix = row[q_col].split('?')[1] + "?"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('?')[0] + "?"
                row[q_col] = row[q_col].strip()
                appedix_dict_1[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_1   

{0: '또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 11: '또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 16: '그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 17: '그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 20: '또한, 피스 하자가 무엇인지 자세히 설명해 주실 수 있나요?',
 22: '또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?',
 23: '또한, 페인트가 남으면 어디에 보관하는 게 좋을까요?',
 24: '또한, 유광 자기질 타일은 얼마나 오랫동안 사용할 수 있을까요?',
 25: '그리고 스탠드조명을 선택할 때 주의할 점이 있을까요?',
 26: '그리고 불량 도배지를 사용할 경우 도배지가 얼마나 오랫동안 버틸까요?',
 27: '그리고 실크벽지의 교체 주기는 얼마나인가요?',
 28: '그리고 철골구조의 장점은 무엇인가요?',
 29: '그리고 조적식 구조란 무엇인가요?',
 33: '또한, 액체방수공사는 무엇을 하는 것인가요?',
 35: '그리고 장판이 남을 때 어떻게 처리해야 하나요?',
 38: '또한, 콘크리트 벽에 구멍을 뚫는 방법에는 어떤 도구나 기술을 사용해야 하나요?',
 39: '그리고 인테리어에 가장 많이 사용되는 도배재료는 무엇인가요?',
 42: '그리고 복도나 협소한 공간을 확장시키기 위해 가장 효과적인 방법이 무엇일까요?',
 48: '또한, 배관공사 시 통기구를 설치해야 하는 이유가 무엇인가요?',
 49: '또한, 규산질계 침투성 도포 방수공사는 어떤 방식으로 이루어지나요?',
 50: '또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?',
 54: '그리고 기둥-보 구조 방식은 무엇을 의미하나요?',
 56: '그리고 새집증후군을 예방하는 데 가장 효과적인 방법은 무엇인가요?',
 59: '또한, 옥상 방수용 탄성 에멀전 페인트를 사용하는 장점은 무엇인가요?',
 63: 

In [7]:

appedix_dict_2 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('그리고') > 0 and row[q_col].count('이를') == 0:
                appendix = row[q_col].split('그리고')[1]
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('그리고')[0]
                row[q_col] = row[q_col].strip()
                appedix_dict_2[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_2     

{6: '합지벽지의 어떤 단점이 있나요?',
 15: '페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 47: '습도가 높을 때 곰팡이가 어떻게 발생하는지 자세히 알고 싶습니다.',
 55: '도배지가 남으면 어떻게 처리해야 하나요?',
 64: '소화기 종류에는 어떤 것들이 있는지 알려주세요.',
 86: '토목이 무엇인지 설명해줘.',
 93: '셀룰로오스의 단점에 대해 설명해주세요.',
 94: '반려동물을 위한 바닥재에는 어떤 종류가 있는지 알려주세요.',
 98: '밀풀 사용 시 주의할 점은 무엇인가요?',
 99: '통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?',
 108: '펫테리어가 무엇인지 자세히 알려주세요.',
 119: '철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.'}

In [8]:
appedix_dict_3 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('또한') > 0:
                appendix = row[q_col].split('또한')[1] + "또한"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('또한')[0] + "또한"
                row[q_col] = row[q_col].strip()
                appedix_dict_3[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()

appedix_dict_3

{2: ', 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한',
 32: ', 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한',
 45: ', 낡은 목재 가구의 흠집을 숨기는 방법을 알려주세요.또한',
 52: ', 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한',
 72: ', 강화마루의 장점은 무엇입니까?또한',
 89: ', 아이소핑크의 장점은 무엇인가요?또한'}

In [9]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

model = PeftModel.from_pretrained(
    model,    # 프롬프트 튜닝에 사용될 기본 모델
    "./kko_lora_8_2_epoch/",     # Peft 모델이 저장된 위치
    is_trainable = False 
)

In [10]:
model.to(device)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTJForCausalLM(
      (transformer): GPTJModel(
        (wte): Embedding(64512, 4096)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-27): 28 x GPTJBlock(
            (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): GPTJAttention(
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
               

In [11]:
# from transformers import AutoModelForSeq2SeqLM+ 
# from peft import PeftModel, PeftConfig 

# # peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
# # config = PeftConfig.from_pretrained(peft_model_id) 
# # model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
# # model = PeftModel.from_pretrained(model, peft_model_id)
# # tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# model = model.to(device)  
# model.eval() 
# inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")   
# with torch.no_grad():     
#     outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)  
#     print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])  'complaint'


In [12]:

# Inference를 위한 test.csv 파일 로드
# test = pd.read_csv('./data/test.csv')
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
file_path = "result_kko.txt"
total_preds = []
# '질문' 컬럼의 각 질문에 대해 답변 생성
# for idx,model in enumerate(model_list):
preds = []
idx = 0
for test_question in tqdm(test_df['질문']):    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')
    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=700,
            min_length=20,
            temperature=0.9,
            top_k=500,
            top_p=0.9,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=10,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        print(answer_only)
        preds.append(answer_only)
        answer_only = answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1


    # with open(csv_file_path, mode='w') as csv_file:
    #     csv_writer = csv.writer(csv_file)
    #     csv_writer.writerow(answer_only)
# break
# total_preds.append(preds)
# model = model.cpu()
# torch.cuda.empty_cache()


  0%|                                                  | 0/130 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  1%|▎                                         | 1/130 [00:10<23:17, 10.84s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the token

방청 페인트의 주요 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다.


  2%|▋                                         | 2/130 [00:33<38:05, 17.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기기 쉽습니다. 이로 인해 도배지에 녹오염이 발생할 수 있으며, 이로 인해 도배지가 손상될 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있으므로 이때는 누수 부분을 보수하고, 곰팡이가 발생한 도배지 부분을 교체하는 것이 좋습니다. 이러한 작업은 전문가의 도움을 받는 것이 추천됩니다.


  2%|▉                                         | 3/130 [00:35<22:48, 10.77s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

평지붕의 누수를 방지하기 위한 대책으로는 먼저, 실내에서 발생하는 수증기의 양을 최대한 억제해야 합니다. 이를 위해 환기를 적절하게 유지하고, 습기 발생원을 관리하며, 건물 내부의 습도를 적정 수준으로 유지해야 합니다. 또한, 충분한 두께의 단열재를 사용하고, 밀착시공하여 단열재와 단열재 사이의 이음부를 통한 열교를 방지하여 온도와 습도의 변화를 최소화해야 합니다. 이렇게 함으로써 누수가 발생할 가능성을 줄일 수 있습니다.


 99%|███████████████████████████████████████▋| 129/130 [20:08<00:12, 12.02s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


석고수정은 벽을 이루고 있는 석고에 훼손이 생겼을 때 발생하는 현상을 말합니다. 석고수정이 발생한 경우 건물의 소유자나 거주자가 책임을 질 수 있습니다. 해결책으로는 석고수정이 있는 부분을 수정하고 전문가에게 접근하여 정밀한 진단을 받아야 합니다. 특히 습기를 흡수하고 물에 녹는 석고의 성질을 고려하여 제습기를 가동하고 환기를 통해 실내 적정 습도를 유지하는 것이 중요합니다. 추가적으로 건축적인 결함이 있는 경우에는 해당 결함을 해결하고 전문가의 도움을 받아 수정하는 것이 필요합니다.


100%|████████████████████████████████████████| 130/130 [20:14<00:00,  9.34s/it]

카페트의 기대수명은 일반적으로 6년입니다. 다만, 사용 빈도, 청소 방식, 햇빛 노출 정도 등 사용 환경과 관리에 따라 수명이 달라질 수 있으므로 주기적으로 청소 및 관리를 해야 합니다. 또한, 질 좋은 카페트를 구매하고 깔끔하게 관리하여 수명을 연장할 수 있습니다.


In [13]:
file_path = "result_kko_appendix_1.txt"


for idx, test_question in tqdm(appedix_dict_1.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=700,
            min_length=20,
            temperature=0.9,
            top_k=500,
            top_p=0.9,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=10,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )


    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        print(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                   | 0/46 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|▉                                          | 1/46 [00:03<02:35,  3.46s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the token

원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.


  4%|█▊                                         | 2/46 [00:10<03:55,  5.35s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


적절하지 않은 도구로 작업하거나 타공부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있습니다.


  7%|██▊                                        | 3/46 [00:27<07:38, 10.67s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

내진구조란 지진이 발생했을 때 건물이나 시설물이 붕괴되지 않도록 설계된 구조를 의미합니다. 내진벽과 같은 부재를 설치하여 지진파에 대비하여 건물을 보다 안전하게 만드는 것이 내진구조의 핵심입니다. 내진설계를 통해 건물이 지진에 대응할 수 있도록 고려되며, 지진 시 건물의 토양이나 지반에 의해 발생하는 흔들림에도 견딜 수 있도록 설계됩니다.


 98%|█████████████████████████████████████████ | 45/46 [05:26<00:04,  4.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


아파트 도배평수는 일반적으로 분양 평수*2.5로 계산합니다.


100%|██████████████████████████████████████████| 46/46 [05:32<00:00,  7.23s/it]

오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다. 또한 오리지널징크는 가벼우면서도 매우 견고하며 낮은 유지보수 비용과 높은 재활용 가능성을 갖고 있습니다.


In [14]:
file_path = "result_kko_appendix_2.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=700,
            min_length=20,
            temperature=0.9,
            top_k=500,
            top_p=0.9,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=10,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        print(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                   | 0/12 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  8%|███▌                                       | 1/12 [00:04<00:46,  4.25s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the token

합지벽지의 어떤 단점이 있나요?</s>합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하여 금방 오염될 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 모양이 훼손될 수도 있습니다.</s>
지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하여 금방 오염될 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 모양이 훼손될 수도 있습니다.


 17%|███████▏                                   | 2/12 [00:10<00:52,  5.21s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


페인트 상도재의 역할과 사용 방법에 대해 알려주세요.</s>페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.</s>
인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.


 25%|██████████▊                                | 3/12 [00:16<00:53,  5.90s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?</s>통나무구조 방식이란 주로 원형이나 각형의 내력벽으로 구성되며, 나머지 바닥이나 지붕은 2x4 경량 목구조와 유사한 방식으로 조립됩니다. 이 구조는 벽체가 통나무를 쌓는 형태이기 때문에 1층 높이에서 7~12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 따뜻한 분위기를 연출해내기 때문에 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.</s>
나무구조 방식이란 주로 원형이나 각형의 내력벽으로 구성되며, 나머지 바닥이나 지붕은 2x4 경량 목구조와 유사한 방식으로 조립됩니다. 이 구조는 벽체가 통나무를 쌓는 형태이기 때문에 1층 높이에서 7~12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 따뜻한 분위기를 연출해내기 때문에 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.


 92%|██████████████████████████████████████▌   | 11/12 [01:06<00:06,  6.26s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


펫테리어가 무엇인지 자세히 알려주세요.</s>펫테리어는 반려동물과 함께 살아가는 가정을 위해 반려동물의 생활환경과 주거 공간의 인테리어를 고려하여 조화롭게 꾸미는 것을 말합니다. 이를 통해 주거 환경을 아름답게 꾸미면서도 반려동물과의 생활을 보다 즐겁게 할 수 있도록 합니다.</s>
테리어는 반려동물과 함께 살아가는 가정을 위해 반려동물의 생활환경과 주거 공간의 인테리어를 고려하여 조화롭게 꾸미는 것을 말합니다. 이를 통해 주거 환경을 아름답게 꾸미면서도 반려동물과의 생활을 보다 즐겁게 할 수 있도록 합니다.


100%|██████████████████████████████████████████| 12/12 [01:12<00:00,  6.06s/it]

철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.</s>철근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.</s>
근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.


In [15]:
file_path = "result_kko_appendix_3.txt"


for idx, test_question in tqdm(appedix_dict_3.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=700,
            min_length=20,
            temperature=0.9,
            top_k=500,
            top_p=0.9,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=10,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                    | 0/6 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
 17%|███████▎                                    | 1/6 [00:14<01:11, 14.33s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the token

, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한</s>압출법 단열판의 장점은 다음과 같습니다:  1. 내구성: 비드법 단열재에 비해 내구성이 뛰어나며 오랜 시간 동안 사용해도 변형이나 손상이 적습니다.  2. 환경 친화적: 재활용이 가능하기 때문에 환경에 미치는 영향을 최소화할 수 있습니다.  3. 저렴한 가격: 다른 단열재에 비해 비교적 저렴하게 구매할 수 있어 경제적인 이점을 가지고 있습니다.  4. 다양한 디자인과 색상: 다양한 형태와 색상으로 제공되어 다양한 인테리어 스타일에 적합합니다.  5. 유지보수가 쉽고 간편합니다. 필요에 따라 교체할 수 있어 유지보수가 편리합니다.  이러한 장점들로 인해 압출법 보온판은 많은 사람들에게 선호되는 단열재로 평가받고 있습니다.</s>


 33%|██████████████▋                             | 2/6 [00:19<00:35,  8.84s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


, 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한</s>세라믹 타일의 단점으로는 온도 변화에 민감하여 내장용으로만 사용해야 하는 점과 경도가 약해 충격과 마모에 쉽게 영향을 받는다는 것이 있습니다. 이러한 특성으로 인해 바닥과 벽면에서 주로 사용되며, 평평한 표면과 내장장식으로 활용되어야 합니다.</s>


 50%|██████████████████████                      | 3/6 [00:25<00:22,  7.57s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

, 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한</s>속건형 유성 발수제는 다공성이 있는 각종 건축자재의 표면에 도장되어 빗물 등의 침투를 막아줌으로써 건축자재의 색상을 보호, 유지시켜주며 건물의 균열이나 백화현상, 철근의 부식을 방지하여 건축물의 수명을 연장시켜주는 장점이 있습니다.</s>


 83%|████████████████████████████████████▋       | 5/6 [00:37<00:06,  6.66s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:63998 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


, 강화마루의 장점은 무엇입니까?또한</s>강화마루는 저렴한 가격과 시공비로 지은 장점이 있습니다. 또한, 강화마루는 충격과 흠집에 강한 속성을 가지고 있어서 내구성이 좋습니다. 또한 시공, 유지보수, 그리고 철거가 용이하며, 접착 시공 없이 설치할 수 있어서 친환경적이라는 점도 장점으로 꼽힙니다.</s>


100%|████████████████████████████████████████████| 6/6 [00:40<00:00,  6.82s/it]

, 아이소핑크의 장점은 무엇인가요?또한</s>아이소핑크는 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.</s>


In [ ]:
test_df

# source_prefix = "<|source|>"
# cand1_prefix = "<|candidate1|>"
# cand2_prefix = "<|candidate2|>"
# inputs = ["hello!", "I love you!"]
# candidates_A = ["hi!", "I hate you!"]
# candidates_B = ["f**k off!", "I love you, too!"]
# def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
#     ids = []
#     assert len(sources) == len(candidate1s) == len(candidate2s)
#     max_length = source_max_length + 2 * candidate_max_length
#     for i in range(len(sources)):
#         source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
#         candidate_max_length = (max_length - len(source_ids)) // 2
#         candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
#         candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
#         ids.append(source_ids + candidate1_ids + candidate2_ids)
#     encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
#     return encodings

# encodings = tokenize_pair(inputs, candidates_A, candidates_B)
# encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
# outputs = pairrm(**encodings)
# logits = outputs.logits.tolist()
# comparison_results = outputs.logits > 0
# print(logits)
# # [1.9003021717071533, -1.2547134160995483]
# print(comparison_results)